# Polars Tutorial and Demonstration
> ### Jonathan Scofield
#### This notebook will help you set up polars on you computer and query a CSV file. For more information about Polars, please visit the official [website](https://pola.rs/). <br>
#### We will be using public real estate sales data from the State of Connecticut for property valued $2K or more from the year 2001 to 2022.
#### The data used for this project is in the public domain and is can be found [here](https://catalog.data.gov/dataset/real-estate-sales-2001-2018).

## Setup <br>
#### You need to have a version of Python installed equal to or greater than 3.10.
#### Use this command to install Polars:
>pip install 'polars[all]' 

In [ ]:
!pip show polars

# Using the SQL Interface

#### Import required modules.

In [ ]:
# "pl" is the conventional alias for the Polars library
import polars as pl
import os

#### Let's take a look at the size of the CSV file we want to examine.

In [ ]:
# Get the size of the source .csv file

f"{round(os.path.getsize(r'Real_Estate_Sales_2001-2022_GL.csv') / (1024 ** 2), 2)} MB"

#### It is fairly large, so we will scan it to a LazyFrame.

In [ ]:
# Create a LazyFrame that infers schema based on first 1000 rows

df = pl.scan_csv(  # We are scanning, not reading
    "Real_Estate_Sales_2001-2022_GL.csv",
    ignore_errors=False,  # No crash on error
    infer_schema_length=1000,  # Sample size for schema detection
    low_memory=True,  # Memory > speed
    try_parse_dates=True,  # Format dates automatically
)

#### After scanning, we can see the proposed schema from the given sample size:

In [ ]:
df.schema # View the inferred schema

#### Because this is a LazyFrame, Polars can't return the count of rows.

In [ ]:
df.select(pl.count())  # This won't return anything on a LazyFrame

## Querying the Data

#### We can query the data as if it was a SQL database using the following syntax:

In [ ]:
# Count records for each year

select_df = pl.SQLContext(register_globals=True).execute(
    """ 
   SELECT
        "List Year",
        count("List Year") as "Record Count"
    FROM 
        df
    GROUP BY "List Year"
    ORDER BY "List Year"
    """
)

#### To view the data, we must call the **collect()** method.

In [ ]:
select_df.collect(streaming=True)  # Perform query and load into memory

#### We can perform most basic SQL queries on the data. Let's try some string manipulation:

In [ ]:
# String manipulation with filtering

pl.SQLContext(register_globals=True).execute(
    """ 
   SELECT
       "Town",
       "Residential Type",
       upper(trim("Town")) + '-' + upper(trim("Residential Type")) as "New Column",
       "Assessed Value"
    FROM 
        df
    WHERE 
        "List Year" = 2021 AND 
        "Assessed Value" > 1000000 
        AND "Residential Type" IS NOT NULL
    ORDER BY 
        "Assessed Value" DESC
   LIMIT 5
    """
).collect(streaming=True)

In [ ]:
# String manipulation with calculated column

highest_premium_df = (
    pl.SQLContext(register_globals=True)
    .execute(
        """ 
   SELECT
        TRIM(UPPER("Town")),
        "Address",
       ("Sale Amount" - "Assessed Value") AS "Premium"
    FROM 
        df
    WHERE 
        "List Year" = 2021 AND 
        "Town" IS NOT NULL
    ORDER BY
       "Premium" DESC
    LIMIT 1
    """
    )
    .collect(streaming=True)
)

highest_premium_df

# Using the Polars Expression Syntax

#### The Polars Expression Syntax will be immediately familiar to anyone who has worked with PySpark. It offers additional granularity and a "Pythonic" syntax for manipulating data.

## Getting Data Profile

In [ ]:
# Descriptive statistics for entire DataFrame

df.describe()

## Basic Selecting and Filtering

#### Basic Selecting

In [ ]:
# Select number of rows in 'Serial Number' column

df.select(pl.col("Serial Number").count()).collect()

#### Selecting with Sort and Limit

In [ ]:
# Select and order by 'Serial Number'

df.select(["Serial Number", "Assessor Remarks"]).limit(5).sort(
    by="Serial Number"
).collect()

#### Select with Filter

In [ ]:
# Filter out null 'Assessor Remarks'

df.select(["Serial Number", "Assessor Remarks"]).filter(
    pl.col("Assessor Remarks").is_not_null()
).limit(5).sort(by="Serial Number").collect()

#### Select with Transformation and Filter

In [ ]:
# Recreate the sales ratio with a column-level calculation

df.select(
    pl.col("List Year").alias("Year"),
    (pl.col("Assessed Value") / pl.col("Sale Amount")).alias("Sales Ratio"),
).filter((pl.col("Year") >= 2020) & (pl.col("Sales Ratio") >= 0.8)).sort(
    by=["Year", "Sales Ratio"], descending=[False, True]
).collect()

## Adding/Transforming Columns

#### Transforming Existing Columns

In [ ]:
# Coalesce remarks and default to "N/A"

df.with_columns(
    pl.coalesce(pl.col(["Assessor Remarks", "OPM remarks"]), pl.lit("N/A")),
    pl.coalesce(pl.col(["OPM remarks", "Assessor Remarks"]), pl.lit("N/A")),
).limit(5).collect()

#### Creating a New Column

In [ ]:
# Create a new column of the struct type

df.filter(
    (pl.col("Assessor Remarks").is_not_null()) & (pl.col("OPM remarks").is_not_null())
).with_columns(
    pl.struct(pl.col(["Assessor Remarks", "OPM remarks"])).alias("Combined Remarks")
).limit(
    5
).collect()

#### Casting

In [ ]:
# Cast a string column to Polars date type

df.select(
    pl.col("Date Recorded").str.strptime(pl.Date, r"%m/%d/%Y"),
    pl.col("Assessed Value").cast(pl.Int64),
).collect()

## Joins

#### SQL-like Joins

In [ ]:
# Sample data
df1 = df.filter(pl.col("List Year") == 2020)
df2 = df.filter(pl.col("List Year") == 2019)

# Inner join
df1.join(df2, on="Address", how="inner").collect()

#### Union All

In [ ]:
# Concatenation of previous DataFrames

pl.concat([df1, df2], how="diagonal_relaxed").collect()

#### Union

In [ ]:
# Concatenation while enforcing uniqueness on the 'Address' column

pl.concat([df1, df2], how="diagonal_relaxed").unique("Address").collect()

## Aggregations

#### Simple Grouping

In [ ]:
# Multi-index grouping with basic aggregations

df.group_by(["List Year", "Town"]).agg(
    [
        pl.col("Assessed Value").mean().alias("Avg Assessed Value"),
        pl.col("Sale Amount").mean().alias("Avg Sale Amount"),
        pl.col("Assessed Value").median().alias("Median Assessed Value"),
        pl.col("Sale Amount").median().alias("Median Sale Amount"),
    ]
).sort(by=["List Year", "Avg Assessed Value", "Avg Sale Amount"]).collect()

#### Grouping Without Aggregating

In [ ]:
# Creates a list if no aggregation is specified

list_df = (
    df.group_by(["List Year", "Town"])
    .agg(
        [
            pl.col("Assessed Value").alias("Assessed Values"),
            pl.col("Sale Amount").alias("Sale Amounts"),
        ]
    )
    .collect()
)

list_df

#### Using Non-Aggregating Groups to Limit Results by Group

In [ ]:
# List types can be sorted, sliced, and exploded to achieve a "top nth" result

list_df.with_columns(
    pl.col("Assessed Values").list.sort(descending=True).list.slice(0, 4),
    pl.col("Sale Amounts").list.sort(descending=True).list.slice(0, 4),
).explode(pl.col("Assessed Values"), pl.col("Sale Amounts"))

## Window Operations

In [ ]:
# Create a DataFrame for the window

window_data = (
    df.select(["List Year", "Address", "Assessed Value"])
    .filter(pl.col("Address") == "1 CEDAR ST")
    .unique()
    .collect()
    .sort("List Year")
)

window_data

#### Using Over()

In [ ]:
# Multi-window operation - rank is over all the rows while the second ranks by year

window_data.with_columns(
    pl.col("Assessed Value").rank("dense", descending=True).alias("Total Rank"),
    pl.col("Assessed Value")
    .rank("dense", descending=True)
    .over("List Year")
    .alias("Yearly Rank"),
)

#### Using Rolling()

In [ ]:
# Create example df

rolling_data = (
    df.select(
        [
            pl.col("Date Recorded").str.strptime(pl.Date, r"%m/%d/%Y"),
            pl.col("Address"),
            pl.col("Assessed Value").cast(pl.Int64),
        ]
    )
    .filter(pl.col("Date Recorded") > pl.date(2020, 1, 1))
    .collect()
    .unique("Date Recorded")
    .sort("Date Recorded")
    .limit(10)
)

rolling_data

In [ ]:
# 3-day rolling mean, max and min

rolling_data.with_columns(
    avg_Val=pl.mean("Assessed Value").rolling(
        index_column="Date Recorded", period="3d"
    ),
    min_Val=pl.min("Assessed Value").rolling(index_column="Date Recorded", period="3d"),
    max_Val=pl.max("Assessed Value").rolling(index_column="Date Recorded", period="3d"),
)

## Pivots and Melts

#### Pivot 

In [ ]:
# Create sample data by grouping median 'Sale Amount' by 'List Year', 'Property Type'

pivot_sample = (
    df.filter(pl.col("Property Type").is_not_null())
    .group_by(["List Year", "Property Type"])
    .agg(pl.col("Sale Amount").median().alias("Median Sale Amount"))
    .collect()
)

pivot_sample

In [ ]:
# Pivot 'Property' Type into columns

pivot_sample.pivot(
    columns="Property Type",
    index="List Year",
    values="Median Sale Amount",
    aggregate_function="mean",
    sort_columns=True,
).sort("List Year")

#### Melt

In [ ]:
# Create sample DataFrame
melt_sample = (
    df.select(["List Year", "Address", "Assessed Value", "Sale Amount"])
    .limit(5)
    .collect()
)

melt_sample

In [ ]:
# Convert into long format by melting 'Assessed Amount' and 'Sale Amount'
melt_sample.melt(
    id_vars=["List Year", "Address"], variable_name="Value Type", value_name="Amount"
).sort("Address")